# 카드 거래 데이터 분석 노트북

Spark ETL 파이프라인의 결과를 탐색하고 시각화하는 노트북입니다.

## 분석 항목
1. 데이터 개요 및 기본 통계
2. 카테고리별 거래 분포
3. 시간대별 거래 패턴
4. 월별 트렌드
5. 지역별 분석
6. 가맹점 매출 TOP

In [ ]:
# 필요 라이브러리 설치
# !pip install pyspark pandas matplotlib seaborn

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, sum as spark_sum, avg, month, hour,
    date_format, to_timestamp, year, dayofweek,
    round as spark_round, desc
)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (Mac)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

plt.style.use('seaborn-v0_8-whitegrid')

# Spark 세션 생성
spark = SparkSession.builder \
    .appName("CardTransactionAnalysis") \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print(f"Spark 버전: {spark.version}")

## 1. 데이터 로드 및 기본 통계

In [ ]:
# 데이터 로드
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../data/card_transactions.csv")

# 타임스탬프 변환
df = df.withColumn("transaction_ts", to_timestamp(col("transaction_date"), "yyyy-MM-dd HH:mm:ss"))
df = df.withColumn("txn_month", month(col("transaction_ts")))
df = df.withColumn("txn_hour", hour(col("transaction_ts")))
df = df.withColumn("txn_date", date_format(col("transaction_ts"), "yyyy-MM-dd"))
df = df.withColumn("txn_dow", dayofweek(col("transaction_ts")))

# 승인 건만
df_approved = df.filter(col("approval_status") == "approved")

print(f"전체 레코드: {df.count():,}")
print(f"승인 건수: {df_approved.count():,}")
print(f"\n스키마:")
df.printSchema()
print("\n기본 통계:")
df_approved.describe("amount").show()

## 2. 카테고리별 거래 분포

In [ ]:
# 카테고리별 집계
cat_stats = df_approved.groupBy("category") \
    .agg(
        count("*").alias("txn_count"),
        spark_sum("amount").alias("total_amount"),
        spark_round(avg("amount"), 0).alias("avg_amount")
    ) \
    .orderBy(desc("total_amount"))

cat_pd = cat_stats.toPandas()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 거래 건수 파이차트
axes[0].pie(cat_pd['txn_count'], labels=cat_pd['category'], autopct='%1.1f%%', startangle=90)
axes[0].set_title('카테고리별 거래 건수 비율')

# 매출 바차트
axes[1].barh(cat_pd['category'], cat_pd['total_amount'] / 1e8)
axes[1].set_xlabel('총 매출 (억원)')
axes[1].set_title('카테고리별 총 매출')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('../docs/category_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

cat_stats.show(truncate=False)

## 3. 시간대별 거래 패턴

In [ ]:
# 시간대별 집계
hourly = df_approved.groupBy("txn_hour") \
    .agg(
        count("*").alias("txn_count"),
        spark_sum("amount").alias("total_amount")
    ) \
    .orderBy("txn_hour")

hourly_pd = hourly.toPandas()

fig, ax1 = plt.subplots(figsize=(12, 5))

color1 = '#2196F3'
color2 = '#FF5722'

ax1.bar(hourly_pd['txn_hour'], hourly_pd['txn_count'], color=color1, alpha=0.7, label='거래 건수')
ax1.set_xlabel('시간')
ax1.set_ylabel('거래 건수', color=color1)
ax1.set_xticks(range(0, 24))

ax2 = ax1.twinx()
ax2.plot(hourly_pd['txn_hour'], hourly_pd['total_amount'] / 1e6, color=color2, linewidth=2, marker='o', label='총 매출(백만원)')
ax2.set_ylabel('총 매출 (백만원)', color=color2)

plt.title('시간대별 거래 패턴')
fig.tight_layout()
plt.savefig('../docs/hourly_pattern.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. 월별 트렌드

In [ ]:
# 월별 집계
monthly = df_approved.groupBy("txn_month") \
    .agg(
        count("*").alias("txn_count"),
        spark_sum("amount").alias("total_amount"),
        spark_round(avg("amount"), 0).alias("avg_amount")
    ) \
    .orderBy("txn_month")

monthly_pd = monthly.toPandas()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 월별 거래 건수
axes[0].bar(monthly_pd['txn_month'], monthly_pd['txn_count'], color='#4CAF50')
axes[0].set_xlabel('월')
axes[0].set_ylabel('거래 건수')
axes[0].set_title('월별 거래 건수')
axes[0].set_xticks(range(1, 13))

# 월별 총 매출
axes[1].plot(monthly_pd['txn_month'], monthly_pd['total_amount'] / 1e8, marker='o', linewidth=2, color='#FF9800')
axes[1].set_xlabel('월')
axes[1].set_ylabel('총 매출 (억원)')
axes[1].set_title('월별 총 매출 추이')
axes[1].set_xticks(range(1, 13))
axes[1].grid(True)

plt.tight_layout()
plt.savefig('../docs/monthly_trend.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. 지역별 분석

In [ ]:
# 지역별 집계
regional = df_approved.groupBy("region") \
    .agg(
        count("*").alias("txn_count"),
        spark_sum("amount").alias("total_amount"),
        spark_round(avg("amount"), 0).alias("avg_amount")
    ) \
    .orderBy(desc("total_amount"))

regional_pd = regional.toPandas()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 지역별 거래 건수
axes[0].barh(regional_pd['region'], regional_pd['txn_count'], color='#9C27B0')
axes[0].set_xlabel('거래 건수')
axes[0].set_title('지역별 거래 건수')
axes[0].invert_yaxis()

# 지역별 평균 거래금액
axes[1].barh(regional_pd['region'], regional_pd['avg_amount'], color='#00BCD4')
axes[1].set_xlabel('평균 거래금액 (원)')
axes[1].set_title('지역별 평균 거래금액')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('../docs/regional_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. 가맹점 매출 TOP 15

In [ ]:
# 가맹점별 집계
merchant = df_approved.groupBy("merchant", "category") \
    .agg(
        count("*").alias("txn_count"),
        spark_sum("amount").alias("total_amount")
    ) \
    .orderBy(desc("total_amount")) \
    .limit(15)

merchant_pd = merchant.toPandas()

fig, ax = plt.subplots(figsize=(12, 6))

colors = plt.cm.Set3(range(len(merchant_pd)))
bars = ax.barh(
    merchant_pd['merchant'] + ' (' + merchant_pd['category'] + ')',
    merchant_pd['total_amount'] / 1e8,
    color=colors
)
ax.set_xlabel('총 매출 (억원)')
ax.set_title('가맹점 매출 TOP 15')
ax.invert_yaxis()

for bar, count_val in zip(bars, merchant_pd['txn_count']):
    ax.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
            f' {count_val:,}건', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../docs/merchant_ranking.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. 요약

In [ ]:
# 전체 요약
total_count = df_approved.count()
total_amount = df_approved.agg(spark_sum("amount")).collect()[0][0]
unique_cards = df_approved.select("card_no").distinct().count()
unique_merchants = df_approved.select("merchant").distinct().count()

print("=" * 50)
print("카드 거래 데이터 분석 요약")
print("=" * 50)
print(f"총 승인 거래 건수 : {total_count:,}")
print(f"총 거래 금액      : {total_amount:,}원 ({total_amount/1e8:.1f}억원)")
print(f"고유 카드 수      : {unique_cards:,}")
print(f"고유 가맹점 수    : {unique_merchants:,}")
print(f"건당 평균 금액    : {total_amount // total_count:,}원")
print("=" * 50)

In [ ]:
# Spark 세션 종료
spark.stop()
print("분석 완료. Spark 세션 종료.")